In [ ]:
# create array of (timesteps, num_samples, q_dim)

num_samples = 100_000
qs = np.tile(q[None, :], (num_samples, 1))
ps = np.tile(p[None, :], (num_samples, 1))

paths  = copy.deepcopy(qs[None, :])
all_fs = np.zeros((timesteps, num_samples, 2))

# create MC rollouts producing array (timesteps, num_samples, q_dim)
for i in range(timesteps-1):
    fs = np.random.normal(loc=loc, scale=scale, size=(num_samples, 2))
    gs = - ps / EPSILON + fs

    all_fs[i] = fs

    p_dots = gs
    ps    += p_dots * EPSILON
    # bound the momentum by the maximum momentum
    # ps     = np.sign(ps) * np.minimum(p_max, np.abs(ps))
    
    q_dots = ps / M
    qs    += q_dots * EPSILON
    # perfectly elastic collisions with walls
    qs  = np.maximum(qs, 2 * Q_MIN - qs)
    ps *= np.sign(qs - Q_MIN)
    qs  = np.minimum(qs, 2 * Q_MAX - qs)
    ps *= np.sign(Q_MAX - qs)

    paths = np.append(paths, qs[None, :], axis=0)

# remove the first state because we calculate conditional density
paths = paths[1:]